In [22]:
from bs4 import BeautifulSoup
import requests
import urllib3
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from requests.packages.urllib3.exceptions import InsecureRequestWarning
# 禁用安全请求警告
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)


In [44]:
# 获取所有的url
def getUrls(url, pageSum):
    urls = []
    pns = [str(i * 1) for i in range(1,pageSum+1)]
    for pn in pns:
        url = url + pn
        urls.append(url)
    return urls

In [3]:
# 获取标签为空处理
def just_null(tag):
    if tag is not None:
        return tag.get_text().strip()
    else:
        return '未知'
    
# 解析标签内容
def analysis_data(page,data_list):
    for i,tag in enumerate(data_list):
        print ('第 ',page,' 页 ',' 第 ',i,' 条')
        
        # 职位信息
        job_info = tag.find('div', attrs={'class': 'job-info'})
        job_type = just_null(tag.find('i', attrs={}).find('b', attrs={}))
        title = just_null(job_info.find('h3', attrs={}).find('a', attrs={}))
        compensation = just_null(job_info.find('span', attrs={'class': 'text-warning'}))
        city = just_null(job_info.find('a', attrs={'class': 'area'}))
        education = just_null(job_info.find('span', attrs={'class': 'edu'}))
        work_experience = just_null(job_info.find('span', attrs={'class': ''}))
        
        time = job_info.find('p', attrs={'class': 'time-info clearfix'}).find('time', attrs={}).get('title')
        # 公司信息
        company_info = tag.find('div', attrs={'class': 'company-info nohover'})
        company = just_null(company_info.find('p', attrs={'class': 'company-name'}).find('a', attrs={}))
        company_type = just_null(company_info.find('a', attrs={'class': 'industry-link'}))
        company_link = job_info.find('h3', attrs={}).find('a', attrs={}).get('href')
        feedback_time = just_null(job_info.find('p', attrs={'class': 'time-info clearfix'}).find('span', attrs={}))

        print (job_type,'   ',title,'   ',company)
        
        # 获取公司人数
        if job_type == '猎':
            company_number = '未知'
        else:
            print (company_link)
            http2 = urllib3.PoolManager()
            r2 = http2.request('GET',company_link)
            soup2 = BeautifulSoup(r2.data, 'lxml')
            info_list = soup2.find('ul', attrs={'class':'new-compintro'}).find_all('li', attrs={})
            if len(info_list) < 2:
                company_number = '未知'
            else:
                company_number = just_null(info_list[1])[5:]
        df.loc[len(df.index)+1] = [title,job_type,compensation,city,education,work_experience,feedback_time,time,company,company_type,company_number,company_link]

In [48]:
# base_url = 'https://www.liepin.com/zhaopin/?fromSearchBtn=2&ckid=0c3548af3dfd5fb9&d_=&degradeFlag=0&init=-1&imscid=R000000058&key=python&headckid=0c3548af3dfd5fb9&d_pageSize=40&siTag=I-7rQ0e90mv8a37po7dV3Q%7EfA9rXquZc5IkJpXC-Ycixw&d_headId=9133450e89c4334ba542648a1889de2c&d_ckId=9133450e89c4334ba542648a1889de2c&d_sfrom=search_fp_bar&d_&curPage='
# bsse_url = 'https://www.liepin.com/zhaopin/?fromSearchBtn=2&ckid=0c3548af3dfd5fb9&d_=&degradeFlag=0&init=-1&imscid=R000000058&key=ios&headckid=0c3548af3dfd5fb9&d_pageSize=40&siTag=I-7rQ0e90mv8a37po7dV3Q%7EfA9rXquZc5IkJpXC-Ycixw&d_headId=9133450e89c4334ba542648a1889de2c&d_ckId=9133450e89c4334ba542648a1889de2c&d_sfrom=search_fp_bar&d_&curPage='
# base_url = 'https://www.liepin.com/zhaopin/?ckid=eee75740ae6b9bd9&fromSearchBtn=2&degradeFlag=0&init=-1&key=ios&imscid=R000000058&headckid=eee75740ae6b9bd9&d_pageSize=40&siTag=njBNTo3xt0z6AJkTGYQoqw%7EfA9rXquZc5IkJpXC-Ycixw&d_headId=970a437ae1072c6d0bb02bb3aa32b573&d_ckId=970a437ae1072c6d0bb02bb3aa32b573&d_sfrom=search_fp_bar&d_&curPage='
base_url = 'https://www.liepin.com/zhaopin/?ckid=eee75740ae6b9bd9&fromSearchBtn=2&degradeFlag=0&init=-1&key=u3d&imscid=R000000058&headckid=eee75740ae6b9bd9&d_pageSize=40&siTag=njBNTo3xt0z6AJkTGYQoqw%7EfA9rXquZc5IkJpXC-Ycixw&d_headId=970a437ae1072c6d0bb02bb3aa32b573&d_ckId=970a437ae1072c6d0bb02bb3aa32b573&d_sfrom=search_fp_bar&d_&curPage='
# 获取所有的url
all_url = getUrls(base_url,21)
# all_url = ['https://www.liepin.com/zhaopin/?fromSearchBtn=2&ckid=0c3548af3dfd5fb9&d_=&degradeFlag=0&init=-1&imscid=R000000058&key=python&headckid=0c3548af3dfd5fb9&d_pageSize=40&siTag=I-7rQ0e90mv8a37po7dV3Q%7EfA9rXquZc5IkJpXC-Ycixw&d_headId=9133450e89c4334ba542648a1889de2c&d_ckId=9133450e89c4334ba542648a1889de2c&d_sfrom=search_fp_bar&d_&curPage=1']
# 创建表
df = pd.DataFrame(columns=['title','job_type','compensation','city','education','work_experience','feedback_time','time','company','company_type','company_number','company_link',])
# 开始抓取
for i,url in enumerate(all_url):
    http = urllib3.PoolManager()
    r = http.request('GET',url)
    soup = BeautifulSoup(r.data, 'lxml')
    data_list = soup.find('ul', attrs={'class': 'sojob-list'}).find_all('li', attrs={})
    analysis_data(i,data_list)

第  0  页   第  0  条
企     AI算法开发工程师     享推科技
https://www.liepin.com/job/198903318.shtml
第  0  页   第  1  条
企     AI算法工程师     享推科技
https://www.liepin.com/job/1910966885.shtml
第  0  页   第  2  条
企     AI算法工程师     杭州中天微系统
https://www.liepin.com/job/1910411969.shtml
第  0  页   第  3  条
企     AI算法专家     杭州中天微系统
https://www.liepin.com/job/1910411951.shtml
第  0  页   第  4  条
企     AI深度学习工程师(IOT)     富士康科技集团CNSBG
https://www.liepin.com/job/1911412840.shtml
第  0  页   第  5  条
企     智能技术（AI）工程师     广州盈通电子科技有限公司
https://www.liepin.com/job/1911387928.shtml
第  0  页   第  6  条
企     AI课程设计师(002690)     达内时代科技集团有限公司
https://www.liepin.com/job/1911347880.shtml
第  0  页   第  7  条
企     AI Lab-Web前端工程师-WebGL     好未来教育
https://www.liepin.com/job/1911331016.shtml
第  0  页   第  8  条
企     AI算法开发工程师     平纳科
https://www.liepin.com/job/1910781386.shtml
第  0  页   第  9  条
企     AI Lab-U3D工程师     好未来教育
https://www.liepin.com/job/1911303769.shtml
第  0  页   第  10  条
企     搜狗AI云—深度学习研究员—自然语言     搜狗
https://www.liepin.com/job/

第  2  页   第  9  条
企     UI/UE设计师     上海怿星电子科技有限公司
https://www.liepin.com/job/1911351248.shtml
第  2  页   第  10  条
企     应用工程师     深圳市前海金交资本控股有限公司
https://www.liepin.com/job/1911096481.shtml
第  2  页   第  11  条
企     平面设计主管     天津华玮摄影有限公司
https://www.liepin.com/job/1911349453.shtml
第  2  页   第  12  条
企     咨询经理     第四范式(北京)技术有限公司
https://www.liepin.com/job/1911348734.shtml
第  2  页   第  13  条
企     平面设计     天津华玮摄影有限公司
https://www.liepin.com/job/1911346728.shtml
第  2  页   第  14  条
企     生态/环境规划师     棕榈生态城镇科技(上海)有限公司
https://www.liepin.com/job/199782999.shtml
第  2  页   第  15  条
企     软件UI工程师     常州金姆健康科技有限公司
https://www.liepin.com/job/1911342187.shtml
第  2  页   第  16  条
企     PR专员     广东美至简网络科技有限公司
https://www.liepin.com/job/1911338232.shtml
第  2  页   第  17  条
企     手绘设计师     句句兽宠物食品
https://www.liepin.com/job/1911331751.shtml
第  2  页   第  18  条
企     眼科技术经理（专职医学顾问）     北京郁金香伙伴科技有限公司
https://www.liepin.com/job/1910178232.shtml
第  2  页   第  19  条
企     pyth开发工程师     云思创智
https://www.liepin.co

第  4  页   第  19  条
企     手绘设计师     句句兽宠物食品
https://www.liepin.com/job/1911331751.shtml
第  4  页   第  20  条
企     眼科技术经理（专职医学顾问）     北京郁金香伙伴科技有限公司
https://www.liepin.com/job/1910178232.shtml
第  4  页   第  21  条
企     pyth开发工程师     云思创智
https://www.liepin.com/job/1911330196.shtml
第  4  页   第  22  条
企     语义分析研究员     索答信息
https://www.liepin.com/job/197794111.shtml
第  4  页   第  23  条
企     中文知识图谱构建研究员     索答信息
https://www.liepin.com/job/197794222.shtml
第  4  页   第  24  条
企     视觉传达主管     江门市蓬江区布维记餐饮管理有限公司
https://www.liepin.com/job/198742428.shtml
第  4  页   第  25  条
企     FAE工程师     速感科技(北京)有限公司
https://www.liepin.com/job/1911315217.shtml
第  4  页   第  26  条
企     网页设计     紫苹果国际设计上海
https://www.liepin.com/job/199227904.shtml
第  4  页   第  27  条
企     技术平台-数据产品经理     美团点评
https://www.liepin.com/job/1911311957.shtml
第  4  页   第  28  条
企     产业招商（副）总监     碧桂园集团
https://www.liepin.com/job/197996992.shtml
第  4  页   第  29  条
企     高级后端开发     智言科技(深圳)有限公司
https://www.liepin.com/job/198096125.shtml
第 

KeyboardInterrupt: 

In [49]:
df

,title,job_type,compensation,city,education,work_experience,feedback_time,time,company,company_type,company_number,company_link
1,AI算法开发工程师,企,面议,上海-浦东新区,硕士及以上,3年工作经验,投递后：1个工作日内反馈,2017年12月22日,享推科技,互联网/移动互联网/电子商务,100-499人,https://www.liepin.com/job/198903318.shtml
2,AI算法工程师,企,面议,常州-新北区,统招本科,经验不限,投递后：1个工作日内反馈,2017年12月22日,享推科技,互联网/移动互联网/电子商务,100-499人,https://www.liepin.com/job/1910966885.shtml
3,AI算法工程师,企,面议,杭州,统招本科,3年工作经验,投递后：5个工作日内反馈,2017年12月22日,杭州中天微系统,电子技术/半导体/集成电路,100-499人,https://www.liepin.com/job/1910411969.shtml
4,AI算法专家,企,面议,杭州,统招本科,8年工作经验,投递后：5个工作日内反馈,2017年12月22日,杭州中天微系统,电子技术/半导体/集成电路,100-499人,https://www.liepin.com/job/1910411951.shtml
5,AI深度学习工程师(IOT),企,面议,深圳,本科及以上,3年工作经验,投递后：2个工作日内反馈,2017年12月22日,富士康科技集团CNSBG,电子技术/半导体/集成电路,10000人以上,https://www.liepin.com/job/1911412840.shtml
6,智能技术（AI）工程师,企,12-24万,广州-天河区,本科及以上,1年工作经验,投递后：5个工作日内反馈,2017年12月22日,广州盈通电子科技有限公司,互联网/移动互联网/电子商务,100-499人,https://www.liepin.com/job/1911387928.shtml
7,AI课程设计师(002690),企,12-18万,北京,本科及以上,2年工作经验,投递后：2个工作日内反馈,2017年12月22日,达内时代科技集团有限公司,教育/培训/学术/科研/院校,5000-10000人,https://www.liepin.com/job/1911347880.shtml
8,AI Lab-Web前端工程师-WebGL,企,面议,北京,统招本科,3年工作经验,投递后：2个工作日内反馈,2017年12月22日,好未来教育,教育/培训/学术/科研/院校,5000-10000人,https://www.liepin.com/job/1911331016.shtml
9,AI算法开发工程师,企,12-30万,上海,统招本科,2年工作经验,投递后：1个工作日内反馈,2017年12月22日,平纳科,外包服务,50-99人,https://www.liepin.com/job/1910781386.shtml
10,AI Lab-U3D工程师,企,面议,北京,统招本科,3年工作经验,投递后：2个工作日内反馈,2017年12月22日,好未来教育,教育/培训/学术/科研/院校,5000-10000人,https://www.liepin.com/job/1911303769.shtml


In [6]:
df.to_csv('ios3.csv')

In [294]:
len(df)

206